In [1]:
# from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.metrics import accuracy_score
from sktime.utils.load_data import load_from_tsfile_to_dataframe
import numpy as np
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from tqdm.notebook import tqdm

import logging
logging.basicConfig(filename="1_nn_dtw_results_history.log", level=logging.INFO)

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def preproc_before_knn_dtw(X):
    X = X.applymap(np.array)

    dimensions_lst = []

    for dim in X.columns:
        dimensions_lst.append(np.dstack(list(X[dim].values))[0])

    dimensions_lst = np.array(dimensions_lst)
    X = torch.from_numpy(np.array(dimensions_lst, dtype=np.float64))
    X = X.transpose(0, 2)
    X = X.transpose(1, 2)
    X = F.normalize(X, dim=1)
    X = X.float().numpy()
    return X


def preproc_answers(y):
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    return y

# Run on all datasets

In [ ]:
datasets_directory = "/root/data/Multivariate_ts/"
datasets_names_lst = os.listdir(datasets_directory)

for dataset_name in tqdm(datasets_names_lst[3:]): # start from the 3rd dataset
    logging.info(f'{dataset_name}')
    print(dataset_name)
    
    try:
        X_train, y_train = load_from_tsfile_to_dataframe(datasets_directory\
                                                         + dataset_name + f'/{dataset_name}_TRAIN.ts')
        X_test, y_test = load_from_tsfile_to_dataframe(datasets_directory\
                                                       + dataset_name + f'/{dataset_name}_TEST.ts')
        print('data has been loaded')
        
        X_train = preproc_before_knn_dtw(X_train)
        X_test = preproc_before_knn_dtw(X_test)
        
        y_train = preproc_answers(y_train)
        y_test = preproc_answers(y_test)
        print('data has been preprocessed')
        
        knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='dtw')
        knn_dtw.fit(X_train, y_train)

        test_predictions = knn_dtw.predict(X_test)
        test_accuracy = accuracy_score(test_predictions, y_test)
        logging.info(f"test_accuracy: {test_accuracy}\n")
        print('test_accuracy:',test_accuracy)
        print('\n')
    except:
        print('Error: smth is wrong\n')
        logging.info(f"Error: smth is wrong\n")

CharacterTrajectories
data has been loaded
Error: smth is wrong

SpokenArabicDigits
data has been loaded
Error: smth is wrong

PhonemeSpectra


# One dataset example

In [3]:
# dataset_name = 'InsectWingbeat'
# dataset_name = 'ERing'
# dataset_name = 'Cricket'
dataset_name = 'PenDigits'

datasets_directory = "/root/data/Multivariate_ts/"

X_train, y_train = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TRAIN.ts')
X_test, y_test = load_from_tsfile_to_dataframe(datasets_directory + dataset_name + f'/{dataset_name}_TEST.ts')

In [4]:
X_train = preproc_before_knn_dtw(X_train)
X_test = preproc_before_knn_dtw(X_test)

y_train = preproc_answers(y_train)
y_test = preproc_answers(y_test)

In [ ]:
# knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='dtw')
knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='dtw')
knn_dtw.fit(X_train, y_train)
print('model was fitted')

# train_predictions = knn_dtw.predict(X_train)
# train_accuracy = accuracy_score(train_predictions, y_train)
# print('train_accuracy:',round(train_accuracy,5))

test_predictions = knn_dtw.predict(X_test)
test_accuracy = accuracy_score(test_predictions, y_test)
print('test_accuracy:', round(test_accuracy,5))
logging.info(f"{dataset_name}: {round(test_accuracy,5)}\n")

model was fitted
